In [1]:
%pip install python-dotenv google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()   # loads .env

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("❌ GEMINI_API_KEY not found. Add it to .env")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ Gemini API key setup complete.")


✅ Gemini API key setup complete.


In [3]:
import os
import google.generativeai as genai

# ensure the key is present
api_key = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise RuntimeError("No API key set. Export GEMINI_API_KEY or GOOGLE_API_KEY.")

genai.configure(api_key=api_key)

print("google-generativeai version:", genai.__version__)

# list models and show which support generateContent
models = list(genai.list_models())
print(f"Found {len(models)} models\n")

valid = []
for m in models:
    methods = getattr(m, "supported_generation_methods", []) or []
    if "generateContent" in methods:
        valid.append(m.name)
        print(f"OK  -> {m.name}   (supports generateContent)")
    else:
        print(f"SKIP -> {m.name}   (methods: {methods})")

if not valid:
    raise RuntimeError("No models with generateContent available to your key/SDK.")



/home/snehal/anaconda3/envs/Google-AI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


google-generativeai version: 0.8.5
Found 65 models

SKIP -> models/embedding-gecko-001   (methods: ['embedText', 'countTextTokens'])
OK  -> models/gemini-2.5-pro-preview-03-25   (supports generateContent)
OK  -> models/gemini-2.5-flash-preview-05-20   (supports generateContent)
OK  -> models/gemini-2.5-flash   (supports generateContent)
OK  -> models/gemini-2.5-flash-lite-preview-06-17   (supports generateContent)
OK  -> models/gemini-2.5-pro-preview-05-06   (supports generateContent)
OK  -> models/gemini-2.5-pro-preview-06-05   (supports generateContent)
OK  -> models/gemini-2.5-pro   (supports generateContent)
OK  -> models/gemini-2.0-flash-exp   (supports generateContent)
OK  -> models/gemini-2.0-flash   (supports generateContent)
OK  -> models/gemini-2.0-flash-001   (supports generateContent)
OK  -> models/gemini-2.0-flash-exp-image-generation   (supports generateContent)
OK  -> models/gemini-2.0-flash-lite-001   (supports generateContent)
OK  -> models/gemini-2.0-flash-lite   (sup

In [4]:
import google.generativeai as genai

MODEL_ID = "models/gemini-2.5-flash"   # <-- replace with one you saw listed
model = genai.GenerativeModel(MODEL_ID)

resp = model.generate_content("Hello! Reply briefly.")
print(resp.text)


Hello!


In [5]:
%pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [6]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


In [7]:
import os, textwrap, pathlib, json

pkg = pathlib.Path("my_agent")
pkg.mkdir(exist_ok=True)

# __init__.py – lets ADK import the module
(pkg / "__init__.py").write_text("from . import agent\n", encoding="utf-8")

# agent.py – defines a global 'root_agent' that ADK scans for
agent_code = textwrap.dedent("""
from google.adk.agents import Agent
from google.adk.tools import google_search

# Use a Gemini 2 model (required for google_search tool)
# Swap to 'gemini-2.5-flash' if you want more capability.
root_agent = Agent(
    name="helpful_assistant",
    model="gemini-2.5-flash-lite",
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)
""").lstrip()
(pkg / "agent.py").write_text(agent_code, encoding="utf-8")

print("✅ Wrote my_agent/__init__.py and my_agent/agent.py")

✅ Wrote my_agent/__init__.py and my_agent/agent.py


In [8]:
from dotenv import load_dotenv
import os, pathlib

(env_path := pathlib.Path("my_agent/.env")).write_text(
    "GOOGLE_API_KEY=" + (os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY") or "PUT_YOUR_KEY_HERE") + "\n",
    encoding="utf-8",
)
load_dotenv("my_agent/.env")
assert os.getenv("GOOGLE_API_KEY"), "Add your key in my_agent/.env"
print("✅ my_agent/.env ready")



✅ my_agent/.env ready


In [9]:
import subprocess, time, os, signal, pathlib
from IPython.display import HTML

# Stop any previous server
pid_file = pathlib.Path("adk_server.pid")
if pid_file.exists():
    try:
        os.kill(int(pid_file.read_text().strip()), signal.SIGINT)
    except ProcessLookupError:
        pass
    finally:
        pid_file.unlink(missing_ok=True)

# Important: start from the *parent* that contains 'my_agent/'
proc = subprocess.Popen(
    ["adk", "web", "--host", "127.0.0.1", "--port", "8000"],
    stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, cwd=os.getcwd()
)
pid_file.write_text(str(proc.pid))
time.sleep(1)
display(HTML("<a href='http://127.0.0.1:8000' target='_blank' "
             "style='display:inline-block;padding:8px 14px;background:#1a73e8;color:#fff;"
             "border-radius:6px;text-decoration:none'>Open ADK Web UI ↗</a>"))
print(f"✅ ADK server started (pid={proc.pid}) at http://127.0.0.1:8000")


✅ ADK server started (pid=6237) at http://127.0.0.1:8000
